In [3]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import cv2

def nordstrom(u0, dt, iterations, g_type, K):
    u = u0.astype(np.float64)
    
    # Definition of the conduction function 
    if g_type == 'gaussian':
        g = lambda x: np.exp(-(x / K)**2)
    elif g_type == 'lorentzian':
        g = lambda x: 1 / (1 + (x / K)**2)
    
    # Calculation of the maximum time step 
    max_dt = 1 / (4 * (1 / u.shape[1]**2 + 1 / u.shape[0]**2))
    if dt > max_dt:
        print(f"Too big time step")
        return u
    
    # Iterations of the finite difference scheme
    for _ in range(iterations):
        Dx_plus = np.roll(u, -1, axis=1) - u
        Dx_minus = u - np.roll(u, 1, axis=1)
        Dy_plus = np.roll(u, -1, axis=0) - u
        Dy_minus = u - np.roll(u, 1, axis=0)
        ge = g(Dx_plus)
        gw = g(Dx_minus)
        gs = g(Dy_plus)
        gn = g(Dy_minus)
        sigma = ge + gw + gs + gn
        weight = 5
        alpha = weight * (u0 - u)
        u += dt * (ge * Dx_plus + gw * Dx_minus + gs * Dy_plus + gn * Dy_minus + alpha) / sigma
        
        # Clip the values to [0, 255] range
        u = np.clip(u, 0, 255)
    
    return u.astype(np.uint8)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
import cv2
# Exemple d'utilisation
# - Image directory
imagedir = "/content/drive/MyDrive/Images"

# - Processed image directory
resultdir = "/content/drive/MyDrive/Results"

# - Input image
filename_in = "graf1.png"

# - Output image
filename_out = filename_in.split('.')[0] + "-results.pgm"

u0 = cv2.imread(imagedir + filename_in, cv2.IMREAD_GRAYSCALE)
dt = 0.1
iterations = 100
g_type = 'gaussian'
K = 10

processed_image = nordstrom(u0, dt, iterations, g_type, K)

output_path = resultdir + filename_out
cv2.imwrite(output_path, processed_image)

print(f"The processed image has been saved as {output_path}.")

The processed image has been saved as C:\Users\sabri\Downloads\Lab_imag\Python\Notebooks\Results\graf1-results.pgm.
